# 13.13 Twitter Streaming API
* Your app can receive tweets as they occur in real-time
* Based on the Twitter Statistics page at [InternetLiveStats.com](http://www.internetlivestats.com/twitter-statistics/)
    * over 10,000 tweets per second 
    * approximately 880 million tweets per day
* Most developer accounts are subject to a **tweet cap** — a maximum number of tweets per month that an account’s Twitter apps can acquire using the Twitter APIs
    * 500,000 for Essentials accounts 
    * two million for Elevated accounts
    * academic research and paid accounts can get more

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.13.1 Creating a Subclass of `StreamListener` 
* A stream uses a **persistent** connection to **push** tweets to your app
* Streaming rate varies tremendously, based on search criteria specified with Tweepy **`StreamRule`** objects
* Twitter uses all the `StreamRule`s you set to find tweets, including `StreamRule`s you’ve set previously
* You may want to delete existing `StreamRule`s before creating new ones

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.13.1 Creating a Subclass of `StreamListener` (cont.)
* Create a subclass of Tweepy’s `StreamingClient` class to process the tweet stream
* Tweepy calls the methods on an object of this class as it receives each new tweet (or other message, such as an error) from Twitter
    * `on_connect(self)` is called when your app successfully connects to the Twitter stream
    * `on_respone(self, response)` is called when a response arrives from the Twitter stream—`response` parameter is a Tweepy `StreamResponse` named tuple object containing the tweet data, any expansion objects you requested and more
* `StreamingClient` already defines these and other "on_" methods 
* Override only the methods your app needs
* `StreamingClient` methods
> https://docs.tweepy.org/en/latest/streamingclient.html  

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener`
`StreamingClient` subclass `TweetListener` is defined in `tweetlistener.py`

```python
# tweetlistener.py
"""StreamingClient subclass that processes tweets as they arrive."""
from deep_translator import GoogleTranslator
import tweepy

class TweetListener(tweepy.StreamingClient):
    """Handles incoming Tweet stream."""
```

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener`: `__init__` Method 
* called when you create a new `TweetListener` object
* `bearer_token` is used to authenticate with Twitter
* `limit` parameter is the number of tweets to process
* Line 11: instance variable to track the number of tweets processed so far
* Line 12: constant to store the limit
* `GoogleTranslator` object for translating tweets into English
* Line 17 passes the `bearer_token` to the superclass’s `__init__`

```python
    def __init__(self, bearer_token, limit=10):
        """Create instance variables for tracking number of tweets."""
        self.tweet_count = 0
        self.TWEET_LIMIT = limit
        
        # GoogleTranslator object for translating tweets to English 
        self.translator = GoogleTranslator(source='auto', target='en')

        super().__init__(bearer_token, wait_on_rate_limit=True)  

```

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener`: `on_connect` Method 
* Called when your app successfully connects to the Twitter stream

```python
    def on_connect(self):
        """Called when your connection attempt is successful, enabling 
        you to perform appropriate application tasks at that point."""
        print('Connection successful\n')
```

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener`: `on_response` Method 
* Called by when each tweet arrives
* second parameter is a Tweepy `StreamResponse` named tuple object containing:
    * `data` — the tweet’s attributes
    * `includes` — any requested expansion objects
    * `errors` — any errors that occurred
    * `matching_rules` — `StreamRules` that the returned tweet matched
* This example uses an expansion to include in the `StreamResponse` the user JSON object for each tweet’s sender
    * Twitter also returns user objects for accounts mentioned in the tweet’s text

```python
    def on_response(self, response):
        """Called when Twitter pushes a new tweet to you."""
        
        try:
            # get username of user who sent the tweet
            username = response.includes['users'][0].username
            print(f'Screen name: {username}')
            print(f'   Language: {response.data.lang}')
            print(f' Tweet text: {response.data.text}')

            if response.data.lang != 'en' and response.data.lang != 'und':
                english = self.translator.translate(response.data.text)
                print(f' Translated: {english}')

            print()
            self.tweet_count += 1 
        except Exception as e:
            print(f'Exception occured: {e}')
            self.disconnect()
            
        # if TWEET_LIMIT is reached, terminate streaming
        if self.tweet_count == self.TWEET_LIMIT:
            self.disconnect()
```

* Line 29 gets the sender’s username
    * List element 0 of `response.includes['users']` contains the tweet sender’s user object
    * Subsequent elements would contain accounts mentioned in the tweet
* Lines 30–32 display the tweet sender’s `username`, the tweet’s language (`lang`) and the tweet’s `text`
* If necessary, lines 34–36 translate the tweet to English and display it
* Line 39 increments `self.tweet_count`
* Lines 45–46 determine whether to terminate streaming. 


<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

# 13.13.2 Initiating Stream Processing

In [1]:
import tweepy

import keys

### Creating a TweetListener 
* `StreamingClient` subclass `TweetListener` manages the connection to the Twitter stream and receives and processes the tweets

In [2]:
from tweetlistener import TweetListener

tweet_listener = TweetListener(
    bearer_token=keys.bearer_token, limit=3)

### Redirecting the Standard Error Stream to the Standard Output Stream
* When `StreamingClient` subclass’s `disconnect` method is called to terminate the tweet stream, the method sends the following message to `sys.stderr` which is not synchronized with the standard output stream
> `Stream connection closed by Twitter`
* Sometimes causes the preceding message to be interspersed with other messages that this app sends to the standard output stream
* To prevent this, redirect the standard error stream to the standard output stream

In [3]:
import sys

sys.stderr = sys.stdout

### Deleting Existing Stream Rules
* Twitter uses all the `StreamRule`s you’ve specified previously to filter the tweets it pushes to your app
* Twitter does not automatically remove your `StreamRule`s after you terminate the tweet stream
* If your app filters the tweet stream with different rules each time you run it, you should delete any existing `StreamRule`s before creating new ones

* Get the `StreamRule`s by calling your `StreamingClient`’s `get_rules` method
    * `Response`’s `data` attribute contains a `list` of `StreamRule`s

In [4]:
rules = tweet_listener.get_rules().data

* Get the rule IDs

In [5]:
rule_ids = [rule.id for rule in rules]

* Call `StreamingClient`’s `delete_rules` method with a list of rule IDs to delete
    * response contains a `'summary'` dictionary with information about the number of deleted rules

In [6]:
tweet_listener.delete_rules(rule_ids)    

Response(data=None, includes={}, errors=[], meta={'sent': '2022-09-24T18:17:59.745Z', 'summary': {'deleted': 1, 'not_deleted': 0}})

### Creating and Adding a Stream Rule
* Create a rule to filter the live tweet stream looking for tweets about football
* Then, add the rule
    * `add_rules`’ Response contains a `'summary'` dictionary with information about the `StreamRule` you set and whether it was valid

In [7]:
filter_rule = tweepy.StreamRule('football')

In [8]:
tweet_listener.add_rules(filter_rule)

Response(data=[StreamRule(value='football', tag=None, id='1573738534137221120')], includes={}, errors=[], meta={'sent': '2022-09-24T18:18:01.090Z', 'summary': {'created': 1, 'not_created': 0, 'valid': 1, 'invalid': 0}})

### Starting the Tweet Stream
* `StreamingClient`'s `filter` method begins streaming 
    * `expansions` argument indicates that we’d like the response for each tweet to include the sender’s user JSON object
    * `tweet_fields` argument indicates that the tweet’s language should be included in the responses tweet JSON object

In [ ]:
tweet_listener.filter( 
    expansions=['author_id'], tweet_fields=['lang'])

### Asynchronous vs. Synchronous Streams
* Tweepy supports asynchronous tweet streams by creating a subclass of `AsyncStreamingClient`
* Allows your application to continue executing while your listener waits to receive tweets
* Convenient in GUI applications, so users can continue interacting with other parts of the application while tweets arrive

------
&copy;1992&ndash;2020 by Pearson Education, Inc. All Rights Reserved. This content is based on Chapter 5 of the book [**Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud**](https://amzn.to/2VvdnxE).

DISCLAIMER: The authors and publisher of this book have used their 
best efforts in preparing the book. These efforts include the 
development, research, and testing of the theories and programs 
to determine their effectiveness. The authors and publisher make 
no warranty of any kind, expressed or implied, with regard to these 
programs or to the documentation contained in these books. The authors 
and publisher shall not be liable in any event for incidental or 
consequential damages in connection with, or arising out of, the 
furnishing, performance, or use of these programs.                  